In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

In [74]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()

# TODO: Prepare the few-shot prompt with examples
few_shot_prompt = """
Here is a classification of the review of some movies and their sentiment which is either positive or negative:

Review: The movie was fantastic, with stunning visuals and a great story.
Sentiment: Positive
Review: The acting was top-notch.
Sentiment: Positive
Review: I found the film boring and overly long. Not worth watching.
Sentiment: Negative
Review: The characters were shallow, and the story was predictable.
Sentiment: Negative
"""

# TODO: Define a function to get GPT-2 prediction for a new sentence
def get_gpt2_prediction(sentence):
    prompt = few_shot_prompt + f"Review: {sentence}\nSentiment:"
    
    # Encode with padding
    inputs = tokenizer(prompt, return_tensors='pt', padding=True)
    
    # Pass attention_mask and pad_token_id to generate()
    outputs = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=inputs['input_ids'].shape[1] + 10,
        do_sample=False,
        num_beams=5,
        early_stopping=True,
        pad_token_id=tokenizer.pad_token_id
    )
    
    # Convert generated text to lowercase for more robust matching
    generated_text = tokenizer.decode(outputs[0]).lower()
    #print("\n\n", generated_text, "\n\n")
    generated_text = "review:" + generated_text.split("review:")[-1]
    #print("\n\n", generated_text, "\n\n")

    
    # Post-process to extract the sentiment prediction
    if "positive" in generated_text:
        return "Positive"
    elif "negative" in generated_text:
        return "Negative"
    else:
        return "Unknown"

In [75]:
test_reviews = [
    "I loved the cinematography and the acting was brilliant.",
    "The story was boring and the pacing was way too slow.",
    "A complete masterpiece! I would watch it again.",
    "Terrible movie, very shallow and i cringed all the time. I regret wasting my time on it.",
    'The movie was fantastic, i liked a lot of stuff from it.'
]

# Test each review
for review in test_reviews:
    sentiment = get_gpt2_prediction(review)
    print(f"Review: {review}")
    print(f"Predicted Sentiment: {sentiment}\n")

Review: I loved the cinematography and the acting was brilliant.
Predicted Sentiment: Negative

Review: The story was boring and the pacing was way too slow.
Predicted Sentiment: Negative

Review: A complete masterpiece! I would watch it again.
Predicted Sentiment: Positive

Review: Terrible movie, very shallow and i cringed all the time. I regret wasting my time on it.
Predicted Sentiment: Negative

Review: The movie was fantastic, i liked a lot of stuff from it.
Predicted Sentiment: Negative

